In [1]:
import cv2
import dlib

# load the face detector and shape predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("models/shape_predictor_68_face_landmarks.dat")

In [14]:
image = cv2.imread("images/gambar3.png")
image = cv2.resize(image, (600, 500))
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Detect faces in the grayscale image
rects = detector(gray)

components_setup = {
    'mulut': {
        'object_name': 'Mouth',
        'object_rectangle': {"x_right": 54, "x_left": 48, "y_highest": 52, "y_lowest": 57},
        'pixel_shifting': {"pixel_x": 25, "pixel_y": 5},
        'object_dimension': {'width': 140, 'height': 40}
    },
    # 'mata_kiri': {
    #     'object_name': 'Left Eye',
    #     'object_rectangle': {"x_right": 39, "x_left": 36, "y_highest": 38, "y_lowest": 41},
    #     'pixel_shifting': {"pixel_x": 25, "pixel_y": 25},
    #     'object_dimension': {'width': 90, 'height': 55}
    # },
    # 'mata_kanan': {
    #     'object_name': 'Right Eye',
    #     'object_rectangle': {"x_right": 45, "x_left": 42, "y_highest": 43, "y_lowest": 47},
    #     'pixel_shifting': {"pixel_x": 25, "pixel_y": 25},
    #     'object_dimension': {'width': 90, 'height': 55}
    # },
    'alis': {  # Gabungan alis kanan dan kiri menjadi satu komponen
        'object_name': 'Eyebrows',
        'object_rectangle': {"x_right": 26, "x_left": 17, "y_highest": 18, "y_lowest": 25},
        'pixel_shifting': {"pixel_x": 15, "pixel_y": 15},
        'object_dimension': {'width': 200, 'height': 40}
    },
    # 'hidung_kanan': {
    #     'object_name': 'nose_right',
    #     'object_rectangle': {"x_right": 31, "x_left": 40, "y_highest": 40, "y_lowest": 48},
    #     'pixel_shifting': {"pixel_x": 15, "pixel_y": -25},
    #     'object_dimension': {'width': 70, 'height': 40}
    # },
    # 'hidung_kiri': {
    #     'object_name': 'nose_left',
    #     'object_rectangle': {"x_right": 47, "x_left": 35, "y_highest": 47, "y_lowest": 54},
    #     'pixel_shifting': {"pixel_x": 15, "pixel_y": -25},
    #     'object_dimension': {'width': 70, 'height': 40}
    # }
}

# Loop over the face detections
for rect in rects:
    # Determine the facial landmarks for the face region
    shape = predictor(gray, rect)
    
    for component_key, component in components_setup.items():
        print("\n{}".format(component['object_name']))
        
        # Mengambil koordinat dari landmark
        x_right = shape.part(component['object_rectangle']['x_right']).x
        x_left = shape.part(component['object_rectangle']['x_left']).x
        y_highest = shape.part(component['object_rectangle']['y_highest']).y
        y_lowest = shape.part(component['object_rectangle']['y_lowest']).y

        # Jika ini komponen alis, ambil koordinat paling ekstrim untuk mencakup kedua alis
        if component_key == 'alis':
            # Menghitung width dan height untuk alis gabungan
            width = x_right - x_left
            height = max(y_lowest - y_highest, 10)  # Pastikan tinggi minimal 10 pixel
            
            # Print the width and height
            print("Width: {}, Height: {}".format(width, height))
            
            # Draw landmarks untuk semua titik antara alis kiri dan kanan
            for i in range(17, 27):  # Rentang landmark untuk kedua alis (17-26)
                x = shape.part(i).x
                y = shape.part(i).y
                label = "{}".format(i)
                cv2.circle(image, (x, y), 4, (255, 0, 0), -1)
                cv2.putText(image, label, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1, cv2.LINE_AA)
            
            # Mengambil titik paling kiri dari alis kiri
            left_most_x = shape.part(17).x
            # Mengambil titik paling kanan dari alis kanan
            right_most_x = shape.part(26).x
            # Mengambil titik tertinggi dari kedua alis
            highest_y = min(shape.part(19).y, shape.part(24).y)
            # Mengambil titik terendah dari kedua alis
            lowest_y = max(shape.part(21).y, shape.part(22).y)
            
            # Menggambar persegi panjang yang mencakup kedua alis
            cv2.rectangle(image,
                         (left_most_x - component['pixel_shifting']['pixel_x'],
                          highest_y - component['pixel_shifting']['pixel_y']),
                         (right_most_x + component['pixel_shifting']['pixel_x'],
                          lowest_y + component['pixel_shifting']['pixel_y']),
                         (0, 255, 0),
                         2)
        else:
            # Untuk komponen lain, tetap menggunakan kode yang sama
            width = x_right - x_left
            height = y_lowest - y_highest

            # Print the width and height
            print("Width: {}, Height: {}".format(width, height))

            # Draw the landmarks
            for i in range(component['object_rectangle']['x_left'], component['object_rectangle']['x_right'] + 1):
                x = shape.part(i).x
                y = shape.part(i).y
                label = "{}".format(i)
                cv2.circle(image, (x, y), 4, (255, 0, 0), -1)
                cv2.putText(image, label, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1, cv2.LINE_AA)

            # Draw a rectangle around the ROI
            cv2.rectangle(image,
                         (x_left - component['pixel_shifting']['pixel_x'],
                          y_highest - component['pixel_shifting']['pixel_y']),
                         (x_right + component['pixel_shifting']['pixel_x'],
                          y_lowest + component['pixel_shifting']['pixel_y']),
                         (0, 255, 0),
                         2)

# Show the output image with the face detections + facial landmarks
cv2.imshow("Frame", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Mouth
Width: 32, Height: 20

Eyebrows
Width: 66, Height: 10


In [5]:
import cv2
import dlib
import os
import numpy as np

detector = dlib.get_frontal_face_detector()
# predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")  # Ganti dengan path model landmark Anda
predictor = dlib.shape_predictor("models/shape_predictor_68_face_landmarks.dat")

foldername_join_basepath = "dataset/training_images"  # Folder berisi gambar

output_folder = "output_images"
os.makedirs(output_folder, exist_ok=True)

# Definisi komponen wajah berdasarkan landmark
components_setup = {
    "eyes": {"indices": list(range(36, 48)), "object_dimension": {"width": 50, "height": 20}},
    "nose": {"indices": list(range(27, 36)), "object_dimension": {"width": 30, "height": 40}},
    "mouth": {"indices": list(range(48, 68)), "object_dimension": {"width": 60, "height": 30}}
}

blockSize = 7  # Ukuran blok untuk pemotongan gambar

def crop_component(image, landmarks, indices, component_name, filename):
    points = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in indices])
    x, y, w, h = cv2.boundingRect(points)
    cropped = image[y:y + h, x:x + w]
    
    output_path = os.path.join(output_folder, f"{filename}_{component_name}.png")
    cv2.imwrite(output_path, cropped)
    print(f"Saved {component_name} crop: {output_path}")

# Loop untuk membaca dan memproses gambar
for filename in sorted(os.listdir(foldername_join_basepath)):
    if filename.endswith(".jpg") or filename.endswith(".png"): 
        image_path = os.path.join(foldername_join_basepath, filename)
        image = cv2.imread(image_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        rects = detector(gray)

        for rect in rects:
            landmarks = predictor(gray, rect)
            for component_name, component_info in components_setup.items():
                crop_component(image, landmarks, component_info['indices'], component_name, filename)

print("Proses selesai!")

Proses selesai!


In [4]:
import numpy as np 
from scipy.fftpack import fft2, ifft2, fftshift
import matplotlib.pyplot as plt 

class POC:
    def __init__(self, imgBlockCur, imgBlockRef, blockSize):
        self.imgBlockCur = imgBlockCur
        self.imgBlockRef = imgBlockRef
        self.blockSize = blockSize

    def hannCalc(self):
        window = np.hanning(self.blockSize)
        window = np.dot(window.T, window)

        return window

    def calcPOC(self, block_ref,block_curr ,window, mb_x, mb_y):
        # Menghitung transformasi Fourier dari blok citra saat ini dengan jendela Hanning
        fft_ref  = fft2(np.dot(block_ref, window), (mb_x, mb_y))
        fft_curr = fft2(np.dot(block_curr, window), (mb_x, mb_y))
        # Menghitung korelasi fase antara dua blok citra
        R1  = fft_ref  * np.conj(fft_curr) 
        # Menghitung magnitudo dari hasil korelasi fase
        R2  = abs(R1) 
        # Mengganti nilai-nol dalam R2 dengan nilai yang sangat kecil untuk menghindari pembagian oleh nol
        R2[R2 == 0] = 1e-31 
        # Menghitung korelasi fase normalisasi
        R   = R1/R2 
        # Menghitung invers transformasi Fourier dari hasil korelasi fase normalisasi
        r   = ifft2(R)
        # Menghitung magnitudo dari hasil invers transformasi Fourier
        r   = abs(r) 
        # Menggeser hasil invers transformasi Fourier agar titik nol berada di tengah
        r   = fftshift(r) 
        return r


    def getPOC(self):
        mb_x = self.blockSize  # panjang macroblock
        mb_y = self.blockSize  # lebar macroblock

        # Perhitunggan Hanning Window
        window = self.hannCalc()

        img0 = self.imgBlockCur
        img1 = self.imgBlockRef

        # konversi image float ke int
        cols, rows = img0.shape
        img0 = img0.astype(int)
        img1 = img1.astype(int)

        # menghitung berapa blok yang dihasilkan
        # dengan pembagian width atau height dan dibagi dengan blocksize
        colsY = np.int16(np.floor(cols / mb_y))
        rowsX = np.int16(np.floor(rows / mb_x))

        # inisiasi untuk menyimpan matrik image yang dipecah dalam blok
        BlocksCurr = np.empty((colsY, rowsX), dtype=object)
        BlocksRef = np.empty((colsY, rowsX), dtype=object)

        # untuk mengetahui sisa pixel
        modY = cols % mb_y
        modX = rows % mb_x

        # inisiasi untuk menyimpan nilai poc
        poc = np.zeros((mb_y, mb_x, colsY * rowsX))
        coorAwal = np.zeros((colsY * rowsX,2))
        rect = np.zeros((colsY * rowsX,4))

        nm = 0
        nY = 0
        nYY =1

        # perulangan y dan x dimulai dari 1
        # perulangan ini akan loncat sesuai dengan blocksize yang ditentukan
        # fungsi cols-modY atau rows-modX untuk pembatas, supaya area perulangan tidak melampaui ukuran gambar
        for y in range(0, cols - modY, mb_y):
            nX = 0
            nXX = 1
            for x in range(0, rows - modX, mb_x):

                # untuk menyimpan block array yang di crop sesuai ukuran
                BlocksCurr[nY, nX] = img0[y:y+mb_y, x: x+mb_x]
                BlocksRef[nY, nX]  = img1[y:y+mb_y, x: x+mb_x]

                rect[nm, :] = [x, y, mb_x, mb_y] #untuk membentuk kotak setiap blok

                block_ref = BlocksRef[nY, nX]
                block_curr = BlocksCurr[nY, nX]

                # Perhitungan POC 
                r = self.calcPOC(block_ref, block_curr, window, mb_x, mb_y)
                # menyimpan nilai poc sesuai nomor blok
                poc[:, :, nm] = r

                coorAwal[nm, 0] = nXX * mb_x  # koordinat X mulai
                coorAwal[nm, 1] = nYY * mb_y  # koordinat Y mulai
                nX  += 1
                nXX += 1
                nm  += 1
            nY  += 1
            nYY += 1
        # kembalian nilai
        # poc : untuk penyimpanan nilai poc disetiap blok
        # coorAwal : sebagai koordinat awal penanda batas blok
        # rect : untuk menyimpak penanda kotak x y width height
        return [poc, coorAwal, rect]


In [ ]:

# # Loop over the face detections
# for rect in rects:
#     # Determine the facial landmarks for the face region
#     shape = predictor(gray, rect)
    
#     for component_key, component in components_setup.items():
#         print("\n{}".format(component['object_name']))
        
#         # Mengambil koordinat dari landmark
#         x_right = shape.part(component['object_rectangle']['x_right']).x
#         x_left = shape.part(component['object_rectangle']['x_left']).x
#         y_highest = shape.part(component['object_rectangle']['y_highest']).y
#         y_lowest = shape.part(component['object_rectangle']['y_lowest']).y

#         # Jika ini komponen alis, ambil koordinat paling ekstrim untuk mencakup kedua alis
#         if component_key == 'alis':
#             # Menghitung width dan height untuk alis gabungan
#             width = x_right - x_left
#             height = max(y_lowest - y_highest, 10)  # Pastikan tinggi minimal 10 pixel
            
#             # Print the width and height
#             print("Width: {}, Height: {}".format(width, height))
            
#             # Draw landmarks untuk semua titik antara alis kiri dan kanan
#             for i in range(17, 27):  # Rentang landmark untuk kedua alis (17-26)
#                 x = shape.part(i).x
#                 y = shape.part(i).y
#                 label = "{}".format(i)
#                 cv2.circle(image, (x, y), 4, (255, 0, 0), -1)
#                 cv2.putText(image, label, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1, cv2.LINE_AA)
            
#             # Mengambil titik paling kiri dari alis kiri
#             left_most_x = shape.part(17).x
#             # Mengambil titik paling kanan dari alis kanan
#             right_most_x = shape.part(26).x
#             # Mengambil titik tertinggi dari kedua alis
#             highest_y = min(shape.part(19).y, shape.part(24).y)
#             # Mengambil titik terendah dari kedua alis
#             lowest_y = max(shape.part(21).y, shape.part(22).y)
            
#             # Menggambar persegi panjang yang mencakup kedua alis
#             cv2.rectangle(image,
#                          (left_most_x - component['pixel_shifting']['pixel_x'],
#                           highest_y - component['pixel_shifting']['pixel_y']),
#                          (right_most_x + component['pixel_shifting']['pixel_x'],
#                           lowest_y + component['pixel_shifting']['pixel_y']),
#                          (0, 255, 0),
#                          2)
#         else:
#             # Untuk komponen lain, tetap menggunakan kode yang sama
#             width = x_right - x_left
#             height = y_lowest - y_highest

#             # Print the width and height
#             print("Width: {}, Height: {}".format(width, height))

#             # Draw the landmarks
#             for i in range(component['object_rectangle']['x_left'], component['object_rectangle']['x_right'] + 1):
#                 x = shape.part(i).x
#                 y = shape.part(i).y
#                 label = "{}".format(i)
#                 cv2.circle(image, (x, y), 4, (255, 0, 0), -1)
#                 cv2.putText(image, label, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1, cv2.LINE_AA)

#             # Draw a rectangle around the ROI
#             cv2.rectangle(image,
#                          (x_left - component['pixel_shifting']['pixel_x'],
#                           y_highest - component['pixel_shifting']['pixel_y']),
#                          (x_right + component['pixel_shifting']['pixel_x'],
#                           y_lowest + component['pixel_shifting']['pixel_y']),
#                          (0, 255, 0),
#                          2)

In [2]:
import os
import cv2
import dlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tabulate import tabulate
import pandas as pd
import shutil
np.set_printoptions(threshold=np.inf)
from preprocessing.scarpping_component import extract_component_by_images

In [ ]:
image = cv2.imread("images/gambar1.png")
# image = cv2.imread("images/test-image.jpg")
image = cv2.resize(image, (600, 500))
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

components_setup = {
    'mulut': {
        'object_name': 'Mouth',
        'object_rectangle': {"x_right": 54, "x_left": 48, "y_highest": 52, "y_lowest": 57},
        'pixel_shifting': {"pixel_x": 25, "pixel_y": 5},
        'object_dimension': {'width': 140, 'height': 40}
    },
    # 'mata_kiri': {
    #     'object_name': 'Left Eye',
    #     'object_rectangle': {"x_right": 39, "x_left": 36, "y_highest": 38, "y_lowest": 41},
    #     'pixel_shifting': {"pixel_x": 25, "pixel_y": 25},
    #     'object_dimension': {'width': 90, 'height': 55}
    # },
    # 'mata_kanan': {
    #     'object_name': 'Right Eye',
    #     'object_rectangle': {"x_right": 45, "x_left": 42, "y_highest": 43, "y_lowest": 47},
    #     'pixel_shifting': {"pixel_x": 25, "pixel_y": 25},
    #     'object_dimension': {'width': 90, 'height': 55}
    # },
    'alis': {  # Gabungan alis kanan dan kiri menjadi satu komponen
        'object_name': 'Eyebrows',
        'object_rectangle': {"x_right": 26, "x_left": 17, "y_highest": 18, "y_lowest": 25},
        'pixel_shifting': {"pixel_x": 15, "pixel_y": 15},
        'object_dimension': {'width': 200, 'height': 40}
    },
    # 'hidung_kanan': {
    #     'object_name': 'nose_right',
    #     'object_rectangle': {"x_right": 31, "x_left": 40, "y_highest": 40, "y_lowest": 48},
    #     'pixel_shifting': {"pixel_x": 15, "pixel_y": -25},
    #     'object_dimension': {'width': 70, 'height': 40}
    # },
    # 'hidung_kiri': {
    #     'object_name': 'nose_left',
    #     'object_rectangle': {"x_right": 47, "x_left": 35, "y_highest": 47, "y_lowest": 54},
    #     'pixel_shifting': {"pixel_x": 15, "pixel_y": -25},
    #     'object_dimension': {'width': 70, 'height': 40}
    # }
}

# load model dan shape predictor untuk deteksi wajah
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("models/shape_predictor_68_face_landmarks.dat")
# Deteksi shape muka didalam grayscale image
rects = detector(gray)

# Memproses rects untuk setiap bentuk wajah yang terdeteksi
for rect in rects:
    # Ambil bentuk wajah dalam bentuk shape sesuai dengan model predictor
    shape = predictor(gray, rect)

    # Ambil data blok image component mulut
    data_blocks_image_current_mulut = extract_component_by_images(
        image=image,
        shape=shape,
        frameName='gambarcoba',
        objectName=components_setup['mulut']['object_name'],
        objectRectangle=components_setup['mulut']['object_rectangle'],
        pixelShifting=components_setup['mulut']['pixel_shifting'],
        objectDimension=components_setup['mulut']['object_dimension']
    )

    print(data_blocks_image_current_mulut)

    # # Ambil data blok image component alis
    # data_blocks_image_current_alis = extract_component_by_images(
    #     image=image,
    #     shape=shape,
    #     frameName='gambar-contoh',
    #     objectName=components_setup['alis']['object_name'],
    #     objectRectangle=components_setup['alis']['object_rectangle'],
    #     pixelShifting=components_setup['alis']['pixel_shifting'],
    #     objectDimension=components_setup['alis']['object_dimension']
    # )

    # print(data_blocks_image_current_alis)

    # Bagi beberapa blok misal 5x5 atau 7x7

    # Masukkan POC tapi harus 2 gambar soalnya perbandingan


# Show the output image with the face detections + facial landmarks
# cv2.imshow("Frame", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

[[169 169 169 169 169 169 168 168 168 167 167 167 167 166 163 163 162 161
  159 158 158 155 154 154 152 150 149 149 146 144 144 143 142 140 139 139
  138 138 138 139 139 140 140 139 138 138 138 138 138 138 138 139 140 140
  140 140 140 140 140 142 143 143 143 145 145 145 147 147 148 148 149 150
  150 149 149 150 150 150 149 149 149 148 148 148 148 147 146 146 146 145
  145 145 144 143 143 143 142 141 140 140 139 139 139 139 138 136 136 136
  136 136 136 136 136 136 136 135 134 134 133 133 132 132 132 132 132 132
  132 131 132 132 131 131 131 130 130 129 128 128 127 126 126]
 [169 169 168 168 168 168 168 167 167 166 166 165 165 164 162 162 160 158
  157 157 156 153 152 151 149 147 146 146 143 141 141 140 139 138 138 137
  137 137 137 138 139 139 139 139 139 139 139 139 139 139 139 140 141 141
  141 141 141 141 141 142 143 143 143 144 145 145 146 147 148 148 149 150
  150 149 150 150 150 150 149 149 149 148 148 148 148 147 146 146 145 146
  145 145 145 143 142 142 141 140 140 140 139 139

In [10]:
image = cv2.imread("images/gambar1.png")
# image = cv2.imread("images/test-image.jpg")
image = cv2.resize(image, (600, 500))
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

components_setup = {
    'mulut': {
        'object_name': 'Mouth',
        'object_rectangle': {"x_right": 54, "x_left": 48, "y_highest": 52, "y_lowest": 57},
        'pixel_shifting': {"pixel_x": 25, "pixel_y": 5},
        'object_dimension': {'width': 140, 'height': 40}
    },
    # 'mata_kiri': {
    #     'object_name': 'Left Eye',
    #     'object_rectangle': {"x_right": 39, "x_left": 36, "y_highest": 38, "y_lowest": 41},
    #     'pixel_shifting': {"pixel_x": 25, "pixel_y": 25},
    #     'object_dimension': {'width': 90, 'height': 55}
    # },
    # 'mata_kanan': {
    #     'object_name': 'Right Eye',
    #     'object_rectangle': {"x_right": 45, "x_left": 42, "y_highest": 43, "y_lowest": 47},
    #     'pixel_shifting': {"pixel_x": 25, "pixel_y": 25},
    #     'object_dimension': {'width': 90, 'height': 55}
    # },
    'alis': {  # Gabungan alis kanan dan kiri menjadi satu komponen
        'object_name': 'Eyebrows',
        'object_rectangle': {"x_right": 26, "x_left": 17, "y_highest": 18, "y_lowest": 25},
        'pixel_shifting': {"pixel_x": 15, "pixel_y": 15},
        'object_dimension': {'width': 200, 'height': 40}
    },
    # 'hidung_kanan': {
    #     'object_name': 'nose_right',
    #     'object_rectangle': {"x_right": 31, "x_left": 40, "y_highest": 40, "y_lowest": 48},
    #     'pixel_shifting': {"pixel_x": 15, "pixel_y": -25},
    #     'object_dimension': {'width': 70, 'height': 40}
    # },
    # 'hidung_kiri': {
    #     'object_name': 'nose_left',
    #     'object_rectangle': {"x_right": 47, "x_left": 35, "y_highest": 47, "y_lowest": 54},
    #     'pixel_shifting': {"pixel_x": 15, "pixel_y": -25},
    #     'object_dimension': {'width': 70, 'height': 40}
    # }
}

# load model dan shape predictor untuk deteksi wajah
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("models/shape_predictor_68_face_landmarks.dat")
# Deteksi shape muka didalam grayscale image
rects = detector(gray)

# Memproses rects untuk setiap bentuk wajah yang terdeteksi
for rect in rects:
    # Ambil bentuk wajah dalam bentuk shape sesuai dengan model predictor
    shape = predictor(gray, rect)

    # # Ambil data blok image component mulut
    # data_blocks_image_current_mulut = extract_component_by_images(
    #     image=image,
    #     shape=shape,
    #     frameName='gambarcoba',
    #     objectName=components_setup['mulut']['object_name'],
    #     objectRectangle=components_setup['mulut']['object_rectangle'],
    #     pixelShifting=components_setup['mulut']['pixel_shifting'],
    #     objectDimension=components_setup['mulut']['object_dimension']
    # )

    # print(data_blocks_image_current_mulut)

    # Ambil data blok image component alis
    data_blocks_image_current_alis = extract_component_by_images(
        image=image,
        shape=shape,
        frameName='gambar-contoh',
        objectName=components_setup['alis']['object_name'],
        objectRectangle=components_setup['alis']['object_rectangle'],
        pixelShifting=components_setup['alis']['pixel_shifting'],
        objectDimension=components_setup['alis']['object_dimension']
    )

    print(data_blocks_image_current_alis)

    # Bagi beberapa blok misal 5x5 atau 7x7

    # Masukkan POC tapi harus 2 gambar soalnya perbandingan


# Show the output image with the face detections + facial landmarks
# cv2.imshow("Frame", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

[[[190 245 162]
  [190 245 162]
  [190 245 162]
  [190 245 162]
  [190 245 162]
  [190 245 162]
  [190 245 162]
  [190 245 162]
  [190 245 162]
  [190 245 162]
  [190 245 162]
  [190 245 162]
  [190 245 162]
  [190 245 162]
  [190 245 162]
  [190 245 162]
  [190 245 162]
  [190 245 162]
  [190 245 162]
  [190 245 162]
  [190 245 162]
  [190 245 162]
  [190 245 162]
  [190 245 162]
  [190 245 162]
  [190 245 162]
  [191 245 162]
  [191 245 162]
  [191 245 162]
  [191 245 162]
  [191 245 162]
  [191 245 162]
  [191 245 162]
  [191 245 162]
  [191 245 162]
  [191 245 162]
  [191 245 162]
  [191 245 162]
  [191 245 162]
  [191 244 162]
  [191 244 162]
  [191 244 162]
  [191 244 162]
  [191 244 162]
  [191 244 162]
  [192 244 162]
  [192 244 162]
  [192 244 162]
  [192 244 162]
  [192 244 162]
  [192 244 162]
  [192 244 162]
  [192 244 162]
  [192 244 162]
  [192 244 162]
  [192 244 162]
  [192 244 162]
  [192 244 162]
  [192 244 162]
  [192 244 162]
  [192 244 162]
  [192 244 162]
  [192 2

In [9]:
import cv2
import dlib
import numpy as np

def extract_component_by_images(image, shape, frameName, objectName, objectRectangle, pixelShifting, objectDimension):
    x1 = objectRectangle['x_left'] + pixelShifting['pixel_x']
    x2 = objectRectangle['x_right'] + pixelShifting['pixel_x']
    y1 = objectRectangle['y_highest'] + pixelShifting['pixel_y']
    y2 = objectRectangle['y_lowest'] + pixelShifting['pixel_y']
    
    extracted_component = image[y1:y2, x1:x2]
    resized_component = cv2.resize(extracted_component, (objectDimension['width'], objectDimension['height']))
    return resized_component

def draw_grid(image, grid_size=(7, 7)):
    h, w = image.shape[:2]
    block_h, block_w = h // grid_size[0], w // grid_size[1]
    
    for i in range(grid_size[0] + 1):
        y = i * block_h
        cv2.line(image, (0, y), (w, y), (255, 255, 255), 1)
    
    for j in range(grid_size[1] + 1):
        x = j * block_w
        cv2.line(image, (x, 0), (x, h), (255, 255, 255), 1)
    
    block_num = 1
    for i in range(grid_size[0]):
        for j in range(grid_size[1]):
            x = j * block_w + 5
            y = i * block_h + 20
            cv2.putText(image, str(block_num), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            block_num += 1
    
    return image

image = cv2.imread("images/coba_mulut2.jpg")
if image is None:
    print("Gambar tidak ditemukan atau gagal dimuat!")
    exit()
image = cv2.resize(image, (600, 500))
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("models/shape_predictor_68_face_landmarks.dat")
rects = detector(gray)

for rect in rects:
    shape = predictor(gray, rect)
    
    mouth_component = extract_component_by_images(
        image=image,
        shape=shape,
        frameName='gambarcoba',
        objectName='Mouth',
        objectRectangle={'x_right': 54, 'x_left': 48, 'y_highest': 52, 'y_lowest': 57},
        pixelShifting={'pixel_x': 25, 'pixel_y': 5},
        objectDimension={'width': 140, 'height': 40}
    )
    
    image_with_grid = draw_grid(image)
    cv2.imshow('Grid Image', image_with_grid)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
